# 最適化

モデルとデータを用意したので、モデルを訓練・検証することで、モデルのパラメータの最適化を行っていく。
各エポックでモデル出力を計算し、損失を求める。そして、各パラメータについて損失に対する偏微分値を求める。そのあと、勾配降下法に基づいてパラメータを最適化する。

In [1]:
%matplotlib inline


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork()


100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



モデルの最適化プロセスを制御するためのパラメータであるハイパーパラメータを設定する。

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5


ハイパーパラメータ設定後、訓練で最適化のループを回すことでモデルを最適化する。
最適化ループの１回のイテレーションをエポックという。

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 予測と損失の計算
        pred = model(X)
        loss = loss_fn(pred, y)

        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------
loss: 2.299215 [    0/60000]
loss: 2.289661 [ 6400/60000]
loss: 2.273258 [12800/60000]
loss: 2.271304 [19200/60000]
loss: 2.261314 [25600/60000]
loss: 2.238326 [32000/60000]
loss: 2.233199 [38400/60000]
loss: 2.209578 [44800/60000]
loss: 2.211697 [51200/60000]
loss: 2.185404 [57600/60000]
Test Error: 
 Accuracy: 53.7%, Avg loss: 0.034351 

Epoch 2
-------------------
loss: 2.193689 [    0/60000]
loss: 2.188533 [ 6400/60000]
loss: 2.141954 [12800/60000]
loss: 2.155356 [19200/60000]
loss: 2.124851 [25600/60000]
loss: 2.081151 [32000/60000]
loss: 2.083030 [38400/60000]
loss: 2.028974 [44800/60000]
loss: 2.049610 [51200/60000]
loss: 1.979372 [57600/60000]
Test Error: 
 Accuracy: 59.1%, Avg loss: 0.031257 

Epoch 3
-------------------
loss: 2.016432 [    0/60000]
loss: 1.998917 [ 6400/60000]
loss: 1.898500 [12800/60000]
loss: 1.931816 [19200/60000]
loss: 1.864282 [25600/60000]
loss: 1.795183 [32000/60000]
loss: 1.802856 [38400/60000]
loss: 1.708933 [44800/60000]
